In [ ]:
!pip install fake_useragent

In [ ]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
from fake_useragent import UserAgent

In [ ]:
URL = 'https://www.transfermarkt.co.uk/fc-chelsea/kader/verein/631/saison_id/2023'

In [ ]:
user_agent = UserAgent()

headers = {
    'User-Agent': user_agent.random,
}

response = requests.get(URL, headers=headers)
webpage = response.text

soup = BeautifulSoup(webpage, 'html.parser')

In [ ]:
table = soup.find(name="table", class_="items")

odd_records = table.find_all(name="tr", class_="odd")
even_records = table.find_all(name="tr", class_="odd")

In [ ]:
player_names = []
player_positions = []
player_market_values = []

In [ ]:
def data_scrapper(record):
  player_name = record.find(name="td", class_="hauptlink").find(name="a").text.strip()

  position = record.find_all('td')[4].get_text(strip=True)

  market_value_m = record.find(name="td", class_="rechts hauptlink").get_text()
  market_value = re.search(r'\d+\.\d+', market_value_m).group()

  return player_name, position, market_value

In [ ]:
for record in odd_records:
  player_name, position, market_value = data_scrapper(record)

  player_names.append(player_name) if player_name else None
  player_positions.append(position) if position else None
  player_market_values.append(market_value) if market_value else None

In [ ]:
for record in even_records:
  player_name, position, market_value = data_scrapper(record)

  player_names.append(player_name) if player_name else None
  player_positions.append(position) if position else None
  player_market_values.append(market_value) if market_value else None

In [ ]:
data = {"Player Name":player_names, "Position": player_positions, "Market Value": player_market_values}

In [ ]:
df = pd.DataFrame(data)
df.head()

,Player Name,Position,Market Value
0,Robert Sánchez,Goalkeeper,30.00
1,Marcus Bettinelli,Goalkeeper,1.00
2,Levi Colwill,Centre-Back,55.00
3,Benoît Badiashile,Centre-Back,35.00
4,Trevoh Chalobah,Centre-Back,18.00


In [ ]:
df.shape

(32, 3)

In [ ]:
df.to_csv('players.csv', index=False)